In [1]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import subprocess
from dtw import correlate
import json
import librosa
import IPython.display as ipd
from random import random
import math

MAX_AUDIO_DURATION = 2.5
MAX_SAMPLE_COUNT_PER_ARTIST = 100
MAX_SAMPLE_COMPARISON_COUNT = 100

apppath = os.path.expanduser('/Users/andrusasumets/projects/experiments/pacemaker/')
processedpath = os.path.join(apppath, 'processed/')
soxipath = os.path.join(processedpath, 'soxi/')
wavpath = os.path.join(processedpath, 'wav/')
audiopath = os.path.join(processedpath, 'audio/')
normalizedpath = os.path.join(processedpath, 'normalized/')
dtwpath = os.path.join(processedpath, 'dtw/')
extensionpath = os.path.join(apppath, 'pacemaker WatchKit Extension/data/')
extensionsamplepath = os.path.join(extensionpath, 'samples/')
librarypath = os.path.expanduser('/Volumes/LaCie/freesound-crawler/downloads/')
allowedfileformats = ['wav', 'aiff', 'flac', 'ogg', 'mp3', 'm4a']
audiofileformat = 'm4a'

for p in [processedpath, soxipath, wavpath, audiopath, normalizedpath, dtwpath, extensionpath, extensionsamplepath]:
    if not os.path.exists(p):
        os.makedirs(p)

In [2]:
f = open(apppath + 'artists.txt')
artists = ''.join(f.readlines()).strip().replace('\n', ',').split(',')
print('artists:', len(artists))
print(artists)

artists: 53
['gameaudio', 'egolessdub', 'batchku', 'dneproman', 'walter-odington', 'errorcell', 'drfx', 'adcbicycle', 'potentjello', 'phr4kture', 'rutgermuller', 'sturmankin', 'ianstargem', 'pjcohen', 'the-very-real-horst', 'felix-blume', 'firnwald', 'suicidity', 'veiler', 'ecodtr', 'martian', 'kanito', 'soneproject', 'cormi', 'john-scott', 'kyles', 'rucisko', 'spankmyfilth', 'samplecat', 'schafferdavid', 'galeku', 'jjbbllkk', 'hello-flowers', 'psysofastia', 'growingup', 'adcbicycle', 'ianstargem', 'ecodtr', 'owlstorm', 'nox-sound', 'chaad', 'giomilko', 'carlmartin', 'art6nik', 'unfav', 'deadrobotmusic', 'kreutzkamp', 'staticpony1', 'the-sacha', 'vinjatovix', 'owstu', 'phantastonia', 'cpolcyn']


In [3]:
def get_fileformat(filename):
    return filename.split('.')[len(filename.split('.')) - 1]

all_filenames = [f for f in os.listdir(librarypath)]
fileformats = {}
for filename in all_filenames:
    fileformat = get_fileformat(filename)
    
    if fileformat not in fileformats:
        fileformats[fileformat] = 0
         
    fileformats[fileformat] = fileformats[fileformat] + 1
    
print('fileformats', fileformats)
print('all_filenames:', len(all_filenames))

filenames = [f for f in os.listdir(librarypath) if f.endswith(tuple(allowedfileformats))]
filenames = [f for f in filenames if f.count('__') == 2]
processedfilenames = [f for f in os.listdir(audiopath) if f.endswith(audiofileformat)]

fileformats {'wav': 43017, 'mp3': 2756, 'aiff': 17770, 'flac': 2696, 'ogg': 588, 'm4a': 141, 'crdownload': 7, 'DS_Store': 1}
all_filenames: 66976


In [4]:
def create_or_read_file(command, outfilename):
    if os.path.exists(outfilename):
        f = open(outfilename, "r")
        out = ''.join(f.readlines())
        f.close()
        return out
    else:
        try:
            out = str(subprocess.check_output(command).decode(encoding="utf-8")).strip().replace('\\n', '').replace("'", "")

            with open(outfilename, 'w') as file:
                file.write(out)

            return out
        except Exception as e:
            return 

In [ ]:
def process(command):
    print(command)
    subprocess.check_output(command)

sample_artists = {}
sample_durations = {}
res = []

for filename in filenames:
    try:
        fileartist = filename.split("__")[1].split("__")[0]
        fileid = filename.split('__')[0]
        audiofilename = fileid + '.' + audiofileformat

        #if fileartist not in artists:
            #continue

        if fileartist not in sample_artists:
            sample_artists[fileartist] = 0

        sample_artists[fileartist] = sample_artists[fileartist] + 1

        if sample_artists[fileartist] >= MAX_SAMPLE_COUNT_PER_ARTIST:
            continue

        soxi_out = create_or_read_file(['soxi', '-D', librarypath + filename], soxipath + fileid + '.soxi')
        if not soxi_out:
            continue

        sample_duration = float(soxi_out)
        if sample_duration > MAX_AUDIO_DURATION:
            continue

        if audiofilename not in processedfilenames:
            process(['ffmpeg-normalize', librarypath + filename, '-o', normalizedpath + filename])
            process(['ffmpeg', '-loglevel', 'error', '-i', normalizedpath + filename, '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', wavpath + fileid + '.wav'])
            process(['ffmpeg', '-loglevel', 'error', '-i', normalizedpath + filename, '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', audiopath + audiofilename])
        
        sample_durations[fileid] = sample_duration
        res.append(audiofilename)
    except Exception as e:
        print(e)

filenames = res
print(filenames)
print(len(filenames))
print(sample_artists)

['ffmpeg-normalize', '/Volumes/LaCie/freesound-crawler/downloads/188650__soniktec__freesound-set-marker-15.wav', '-o', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/188650__soniktec__freesound-set-marker-15.wav']


['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/188650__soniktec__freesound-set-marker-15.wav', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/wav/188650.wav']
['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/188650__soniktec__freesound-set-marker-15.wav', '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/audio/188650.m4a']
['ffmpeg-normalize', '/Volumes/LaCie/freesound-crawler/downloads/189537__tinga__turdus-merula-26.wav', '-o', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/189537__tinga__turdus-merula-26.wav']


['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/189537__tinga__turdus-merula-26.wav', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/wav/189537.wav']
['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/189537__tinga__turdus-merula-26.wav', '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/audio/189537.m4a']
['ffmpeg-normalize', '/Volumes/LaCie/freesound-crawler/downloads/345365__jalastram__kick-010.wav', '-o', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/345365__jalastram__kick-010.wav']


['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/345365__jalastram__kick-010.wav', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/wav/345365.wav']
['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/345365__jalastram__kick-010.wav', '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/audio/345365.m4a']
['ffmpeg-normalize', '/Volumes/LaCie/freesound-crawler/downloads/426517__shintaboy__56-piedras.wav', '-o', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/426517__shintaboy__56-piedras.wav']


['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/426517__shintaboy__56-piedras.wav', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/wav/426517.wav']
['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/426517__shintaboy__56-piedras.wav', '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/audio/426517.m4a']
['ffmpeg-normalize', '/Volumes/LaCie/freesound-crawler/downloads/557982__zzztefanos__zzzbeats-miscellaneous-music-fragments-0217.wav', '-o', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/557982__zzztefanos__zzzbeats-miscellaneous-music-fragments-0217.wav']


...

['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/14215__adcbicycle__20.wav', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/wav/14215.wav']
['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/14215__adcbicycle__20.wav', '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/audio/14215.m4a']
['ffmpeg-normalize', '/Volumes/LaCie/freesound-crawler/downloads/264659__michael-klier__bundeswehr-bag-light-heavy-movement-processed-24bit96khz-005.wav', '-o', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/264659__michael-klier__bundeswehr-bag-light-heavy-movement-processed-24bit96khz-005.wav']


['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/264659__michael-klier__bundeswehr-bag-light-heavy-movement-processed-24bit96khz-005.wav', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/wav/264659.wav']
['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/264659__michael-klier__bundeswehr-bag-light-heavy-movement-processed-24bit96khz-005.wav', '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/audio/264659.m4a']
['ffmpeg-normalize', '/Volumes/LaCie/freesound-crawler/downloads/573552__gamedrix974__21.wav', '-o', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/573552__gamedrix974__21.wav']


['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/573552__gamedrix974__21.wav', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/wav/573552.wav']
['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/573552__gamedrix974__21.wav', '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/audio/573552.m4a']
['ffmpeg-normalize', '/Volumes/LaCie/freesound-crawler/downloads/459610__bolkmar__fx-laser-shoot-a.wav', '-o', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/459610__bolkmar__fx-laser-shoot-a.wav']


['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/459610__bolkmar__fx-laser-shoot-a.wav', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/wav/459610.wav']
['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/459610__bolkmar__fx-laser-shoot-a.wav', '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/audio/459610.m4a']
['ffmpeg-normalize', '/Volumes/LaCie/freesound-crawler/downloads/276393__beskhu__cymbesse-4.wav', '-o', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/276393__beskhu__cymbesse-4.wav']


['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/276393__beskhu__cymbesse-4.wav', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/wav/276393.wav']
['ffmpeg', '-loglevel', 'error', '-i', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/276393__beskhu__cymbesse-4.wav', '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', '/Users/andrusasumets/projects/experiments/pacemaker/processed/audio/276393.m4a']
['ffmpeg-normalize', '/Volumes/LaCie/freesound-crawler/downloads/476377__mccaslinmusic__jp-hi-hat-2.wav', '-o', '/Users/andrusasumets/projects/experiments/pacemaker/processed/normalized/476377__mccaslinmusic__jp-hi-hat-2.wav']


In [ ]:
def create_or_read_file(filename, command):
    if os.path.exists(filename):
        f = open(filename, "r")
        out = ''.join(f.readlines())
        f.close()
        return out
    else:
        try:
            out = str(subprocess.check_output(command).decode(encoding="utf-8")).strip().replace('\\n', '').replace("'", "")

            with open(filename, 'w') as file:
                file.write(out)

            return out
        except Exception as e:
            return 

In [ ]:
df = pd.DataFrame()
out_distances = {}
full_distances = []

for leftfilename in filenames:
    leftid = leftfilename.split('.')[0]
    rightfilenames = filenames
    rightfilenames = sorted(rightfilenames, key=lambda x: random())
    
    for righti, rightfilename in enumerate(rightfilenames):
        if (len(full_distances) % 100) == 0 and righti == 0:
            print(str(len(full_distances)) + " / " + str(len(filenames) * MAX_SAMPLE_COMPARISON_COUNT))
            
        if righti >= MAX_SAMPLE_COMPARISON_COUNT:
            continue
        
        if leftfilename == rightfilename:
            continue
            
        rightid = rightfilename.split('.')[0]
            
        try:
            correlation = correlate(wavpath, leftid, rightid, 'wav', dtwpath)
            
            if math.isinf(float(correlation)):
                continue
                
            if math.isnan(float(correlation)):
                continue
            
            df.loc[leftid, rightid] = correlation
            df.loc[rightid, leftid] = correlation

            if leftid not in out_distances:
                out_distances[leftid] = []
            
            sample_duration = sample_durations[leftid]
            out_distances[leftid].append([correlation, sample_duration, int(rightid)])
            full_distances.append([correlation, leftid, rightid])

        except Exception as e:
            pass

for filename in out_distances:
    out_distances[filename] = sorted(out_distances[filename])
    
full_distances = sorted(full_distances)
print(len(full_distances))

In [ ]:
delimiter = '|'
samples = {}
filtered_filenames = [f for f in os.listdir(dtwpath) if f.endswith('.dtw')]

for filename in filtered_filenames:
    filename = filename.split('.dtw')[0]
    left, right = filename.split(delimiter)
    
    if left not in samples:
        samples[left] = []
        
    if right not in samples:
        samples[right] = []
        
    samples[left].append(right)
    samples[right].append(left)

In [ ]:
saved = []

for distance in full_distances:
    leftid = distance[1]
    
    if leftid in saved:
        continue
        
    saved.append(leftid)
    
    source = audiopath + leftid + '.' + audiofileformat
    target = extensionsamplepath + leftid + '.' + audiofileformat
    command = ['cp', source, target]
    process(command)

print(len(saved))

with open(extensionpath + 'distances.json', 'w') as fp:
    json.dump(out_distances, fp)

In [ ]:
plot = sns.heatmap(df, xticklabels=False, yticklabels=False)
handles, labels = plot.get_legend_handles_labels()
plot.legend(reversed(handles), reversed(labels), title='', loc="lower right")
fig = plot.get_figure()
fig.tight_layout()
fig.savefig(extensionpath + "heatmap.png", dpi=1000, facecolor="white")

In [ ]:
def show_audio(comparison):
    distance, leftid, rightid = comparison
    print(distance)

    data, rate = librosa.core.load(audiopath + leftid + '.' + audiofileformat)
    ipd.display(ipd.Audio(data, rate=rate))

    data, rate = librosa.core.load(audiopath + rightid + '.' + audiofileformat)
    ipd.display(ipd.Audio(data, rate=rate))

display_count = 10
for i in range(display_count):
    if (i % 2) == 0:
        show_audio(full_distances[i])
        
for i in range(display_count):
    if (i % 2) == 0:
        show_audio(full_distances[i - display_count])